# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.



In [2]:
!pip install syft
!pip install numpy

In [0]:
import torch as th

In [4]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [6]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [7]:
import syft as sy

W0702 23:42:17.854516 140456370104192 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0702 23:42:17.869269 140456370104192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)  # this line of code create an instance of th with its backend APIs modified with Syft functions

In [9]:
th.tensor([1,2,3,4,5]) # notice that Torch functionalities still behave the same

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob") # creates a virtual worker (a simulaion to interface to Bob machines)

In [11]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr =  x.send(bob) # send this data to bob

In [14]:
bob._objects

{81542683161: tensor([1, 2, 3, 4, 5])}

In [15]:
x_ptr

(Wrapper)>[PointerTensor | me:69569918714 -> bob:81542683161]

In [16]:
x.location

AttributeError: ignored

In [0]:
x.id_at_location

In [0]:
x.id

In [0]:
x.owner

In [0]:
hook.local_worker

In [0]:
x = x.get()
x

In [0]:
bob._objects

# Project: Experience with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# create a second Virtual worker and call it Alice


In [0]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice


In [0]:
# notice what does the send fucntion on two workers return; a multi-pointer
x_ptr

In [0]:
# what does .child on the pointer object return?
# what does .child.child on the pointer object return? 

In [0]:
# try the .get() on the pointer


In [0]:
# try the .get(sum_results=True) on your pointer


# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
x # a pointer to a remote tensor located at bob

In [0]:
y # a pointer to another remote tensor located at bob

In [0]:
z = x + y # treat those tensors (i.e., x and y) as local tensors, but they are actually executed remotely
z

In [0]:
z = z.get()
z

In [0]:
z = th.add(x,y) # we can also use Torch operation on these tensors to be executed remotely
z

In [0]:
z = z.get() # yup! same results!
z

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()
z = z.get()
z

In [0]:
z.backward() 

In [0]:
x = x.get()

In [0]:
x

In [0]:
x.grad

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). 

Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)  
bob = sy.VirtualWorker(hook, id="bob")

bob

In [0]:
# create some toy data for our model
input_data = th.tensor([[1.,1],[0.5,1],[1,0],[0,0]], requires_grad=True).send(bob)
output_data = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
# send the data to bob

In [0]:
# create some linear weights and send them to bob
weights = th.tensor([[0.01],[0.01]], requires_grad = True).send(bob)

In [0]:
# create a linear model and train it on Bob's machine
# remember how to create a linear model? No? :( 
# Here's the main steps:
#    1- find a prediction
#    2- calcualte the loss (a mean square loss)
#    3- backpropogate using the backword() function
#    4- DO NOT forget to clear your gradients after updating the weights
#    weights.data.sub_(weights.grad * lr)

prediction = input_data.mm(weights)
prediction

In [0]:
lr = 0.1

for i in range(10):
  prediction = input_data.mm(weights)
  loss = ((prediction - output_data)**2).mean()
  loss.backward()
  weights.data.sub_(weights.grad * lr)
  weights.grad *= 0

  print(loss.get().data)

# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects() # clear the contents of a remote object

In [18]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [20]:
bob._objects

{14432461464: tensor([1, 2, 3, 4, 5])}

In [0]:
del x  # delete the pointer to the remote object

In [22]:
bob._objects

{}

In [25]:
x.child.garbage_collect_data  # True by default

True

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [27]:
bob._objects

{96116517439: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [29]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [31]:
x

(Wrapper)>[PointerTensor | me:80887835768 -> bob:57197099033]

In [32]:
bob._objects

{57197099033: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [34]:
bob._objects  # some error from Jupyter is casuing this to appear!

{57197099033: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [36]:
bob._objects

{57197099033: tensor([1, 2, 3, 4, 5])}

In [37]:
bob = bob.clear_objects() # erase force 
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [39]:
bob._objects # notice that there is only a single tensor in bob

{99831005766: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [0]:
z = x.get() + y

In [47]:
bob

<VirtualWorker id:bob #objects:0>

In [0]:
alice = sy.VirtualWorker(hook, id="alice")
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [0]:
z = x.get() + y.get()

In [51]:
z

tensor([2, 3, 4, 5, 6])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [58]:
model

Linear(in_features=2, out_features=1, bias=True)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [60]:
def train(iterations=10):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(3.5317)
tensor(0.6122)
tensor(0.2743)
tensor(0.1717)
tensor(0.1135)
tensor(0.0759)
tensor(0.0510)
tensor(0.0346)
tensor(0.0236)
tensor(0.0162)


In [61]:
# nothing was federated up to this point!

import torch as th
import syft as sy

hook = sy.TorchHook(th)  
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")



W0702 23:52:06.378851 140456370104192 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
# create local datasets at Bob and Alice
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [65]:
bob._objects

{189199953: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 16005265611: tensor([[1.],
         [1.]], requires_grad=True)}

In [66]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]
datasets

[((Wrapper)>[PointerTensor | me:6683938492 -> bob:189199953],
  (Wrapper)>[PointerTensor | me:58994451461 -> bob:16005265611]),
 ((Wrapper)>[PointerTensor | me:30257994301 -> alice:74941075892],
  (Wrapper)>[PointerTensor | me:51088810429 -> alice:66807525786])]

In [0]:
def train(iterations=1):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:
          
            print(model)

            # send model to the data
            model = model.send(_data.location)

            # do training on the remote machine
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()
            
            print(model)

            print(loss.get())

In [71]:
train()

Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=2, out_features=1, bias=True)
tensor(4.1600, requires_grad=True)
Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=2, out_features=1, bias=True)
tensor(0.2474, requires_grad=True)


In [69]:
bob._objects

{189199953: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 16005265611: tensor([[1.],
         [1.]], requires_grad=True)}

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [72]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [86]:
bob._objects

{}

In [76]:
alice._objects

{}

In [77]:
y = x + x

KeyError: ignored

In [78]:
y

AttributeError: ignored

In [0]:
bob._objects

{681755396: tensor([1, 2, 3, 4, 5]), 61259427323: tensor([ 2,  4,  6,  8, 10])}

In [0]:
alice._objects

{14700048988: (Wrapper)>[PointerTensor | alice:14700048988 -> bob:61259427323],
 59232059007: (Wrapper)>[PointerTensor | alice:59232059007 -> bob:681755396]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()



<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [0]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
x = x.get() # to get the data back
x

(Wrapper)>[PointerTensor | me:59854104348 -> bob:36602545298]

In [0]:
bob._objects

{36602545298: tensor([1, 2, 3, 4, 5]), 89784608003: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{}

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

# Lesson: Pointer Chain Operations

In [79]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [81]:
bob._objects

{}

In [82]:
alice._objects

{}

In [83]:
x.move(alice)

KeyError: ignored

In [0]:
x

(Wrapper)>[PointerTensor | me:16851411436 -> alice:16851411436]

In [0]:
bob._objects

{}

In [0]:
alice._objects

{16851411436: tensor([1, 2, 3, 4, 5])}

In [0]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

{81869613150: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{99685627431: (Wrapper)>[PointerTensor | alice:99685627431 -> bob:81869613150]}

In [0]:
x

(Wrapper)>[PointerTensor | me:83688287202 -> alice:99685627431]

In [0]:
x.remote_get() # inplace operation

(Wrapper)>[PointerTensor | me:83688287202 -> alice:99685627431]

In [0]:
bob._objects

{}

In [0]:
alice._objects

{99685627431: tensor([1, 2, 3, 4, 5])}

In [0]:
x.move(bob)

(Wrapper)>[PointerTensor | me:83688287202 -> bob:83688287202]

In [0]:
x

(Wrapper)>[PointerTensor | me:83688287202 -> bob:83688287202]

In [0]:
bob._objects

{83688287202: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{}

**Exercise 1:**

- Create a toy dataset
- Create a toy linear model
- Train the dataset using Federated Learning, but do not share the gradiant with the main server, instead use the move function to send the gradients to one of the participating workers, aggregate the model, then send the final model to the main server (i.e., local worker).



---


**Exercise 2:**

While exercise 1 illustrates a good example of preseving privacy in FL, we still had to expose the gradients to one of the participating workers. To avoid this limitation, you need to send the gradiants to a TRUSTED thirdparty (trusted aggregator) who will aggregate the models and then send the final model to the server (local worker). In this way, we assure that none of the participating workers can access the aggregated model!

1. create a dataset for each worker (create two)
2. create a model for each worker and train it remotely on each worker
3. send those two models using the *move()* function to a third worker
4. the third workers aggregates the two models (find their mean)
5. send the aggregated model to the main server (local worker) using the *get()* function



---

To set the weights: `model.weight.set_()`



In [0]:
# Exercise 1
import syft as sy
import torch as th
from torch import nn, optim

In [88]:
hook = sy.TorchHook(th)

W0703 00:20:36.957712 140456370104192 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
# create workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secureWorker = sy.VirtualWorker(hook, id="secureWorker")

In [90]:
# Make each worker aware of the other workers
bob.add_workers([alice,secureWorker])
alice.add_workers([bob,secureWorker])
secureWorker.add_workers([bob, alice])

W0703 00:21:22.907921 140456370104192 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0703 00:21:22.909293 140456370104192 base.py:628] Worker secureWorker already exists. Replacing old worker which could cause                     unexpected behavior
W0703 00:21:22.910720 140456370104192 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0703 00:21:22.911553 140456370104192 base.py:628] Worker secureWorker already exists. Replacing old worker which could cause                     unexpected behavior
W0703 00:21:22.913225 140456370104192 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0703 00:21:22.914354 140456370104192 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secureWorker #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [0]:
# create local datasets at Bob and Alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[0:2].send(alice)
alices_target = target[0:2].send(alice)

In [100]:
# create a linear model at local worker
model = nn.Linear(2,1)
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.3980, -0.6481]], requires_grad=True), Parameter containing:
tensor([-0.1882], requires_grad=True)]


In [0]:
# Send copies of the linear model to alice and bob
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)



In [104]:
bob._objects

{189199953: tensor([[1., 1.],
         [0., 1.]], requires_grad=True),
 6809343046: tensor([1, 2, 3, 4, 5]),
 9443909219: Parameter containing:
 tensor([[ 0.3980, -0.2651]], requires_grad=True),
 16005265611: tensor([[1.],
         [1.]], requires_grad=True),
 19947902485: tensor([1, 2, 3, 4, 5]),
 33047717862: tensor([[0.],
         [0.]], requires_grad=True),
 51430868463: tensor([[0., 0.],
         [0., 1.]], requires_grad=True),
 72484367660: Parameter containing:
 tensor([0.1494], requires_grad=True),
 90480901957: tensor([[ 0.1511],
         [-0.1425]], grad_fn=<AddmmBackward>)}

In [0]:
def train(iterations=1):
  # create a linear model at local worker
  model = nn.Linear(2,1)
  print(list(model.parameters()))
  
  # Send copies of the linear model to alice and bob
  bobs_model = model.copy().send(bob)
  alices_model = model.copy().send(alice)

  bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
  alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)


  # train the models at alice and bob
  # move the models to the third worker
  # aggreegate the models
  # send the model back to the local worker
  for i in range(5):

    # Train Bob's Model
      bobs_opt.zero_grad()
      bobs_pred = bobs_model(bobs_data)
      bobs_loss = ((bobs_pred - bobs_target)**2).sum()
      bobs_loss.backward()

      bobs_opt.step()
      bobs_loss = bobs_loss.get().data
      
      print("bobs_loss :",bobs_loss)

    # Train Alice's Model
      alices_opt.zero_grad()
      alices_pred = alices_model(alices_data)
      alices_loss = ((alices_pred - alices_target)**2).sum()
      alices_loss.backward()

      alices_opt.step()
      alices_loss = alices_loss.get().data
      alices_loss
      
      print("alices_loss :",alices_loss)
      
      
  alices_model.move(secureWorker)
  bobs_model.move(secureWorker)
  
  with th.no_grad():

    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())



In [115]:
train()

[Parameter containing:
tensor([[0.4585, 0.2613]], requires_grad=True), Parameter containing:
tensor([-0.3980], requires_grad=True)]
bobs_loss : tensor(0.1771)
alices_loss : tensor(0.1771)
bobs_loss : tensor(0.0847)
alices_loss : tensor(0.0847)
bobs_loss : tensor(0.0572)
alices_loss : tensor(0.0572)
bobs_loss : tensor(0.0454)
alices_loss : tensor(0.0454)
bobs_loss : tensor(0.0380)
alices_loss : tensor(0.0380)


In [0]:
def train(iterations=1):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:
          
            print(model)

            # send model to the data
            model = model.send(_data.location)

            # do training on the remote machine
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()
            
            print(model)

            print(loss.get())